<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/2025_06_30_LGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!unzip -o "/content/drive/MyDrive/data.zip" -d "/content/data_1"


Archive:  /content/drive/MyDrive/data.zip
  inflating: /content/data_1/sample_submission.csv  
  inflating: /content/data_1/test.csv  
  inflating: /content/data_1/train.csv  


In [16]:
import os

def get_path(filename):
    return "/content/data_1/" + filename


In [4]:
# SMOTE 설치
!pip install -U imbalanced-learn
!pip install lightgbm optuna
!pip install -q lightgbm catboost xgboost



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.6 MB/s eta 0:00:00


In [5]:
!pip install --upgrade lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


In [6]:
import lightgbm as lgb
print(lgb.__version__)

4.6.0


In [17]:
# ✅ 라이브러리 설치
!pip install rdkit-pypi optuna lightgbm -q

In [23]:

# ✅ 라이브러리 임포트
import zipfile, os, warnings
import pandas as pd
import numpy as np
import optuna
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import early_stopping, log_evaluation
import lightgbm as lgb

warnings.filterwarnings("ignore")

# ✅ 경로 설정 및 압축 해제
zip_path = "/content/drive/MyDrive/data.zip"
extract_path = "/content/data_1"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"✅ 압축 해제 완료: {extract_path}")

# ✅ 경로 함수
def get_path(filename):
    return os.path.join(extract_path, filename)

# ✅ 데이터 로딩 + 컬럼 공백 제거
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

train.columns = train.columns.str.strip()
test.columns = test.columns.str.strip()
submission.columns = submission.columns.str.strip()

# ✅ SMILES → 분자 특성 변환 함수
def smiles_to_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [np.nan] * 7
    return [
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.TPSA(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.RingCount(mol)
    ]

# ✅ 학습 데이터 Feature Engineering
desc_list = train["Canonical_Smiles"].apply(smiles_to_descriptors)
desc_df = pd.DataFrame(desc_list.tolist(), columns=[
    "MolWt", "MolLogP", "HDonors", "HAcceptors", "TPSA", "RotBonds", "Rings"
])
desc_df["Inhibition"] = train["Inhibition"]
desc_df = desc_df.dropna()

# ✅ 테스트 데이터 Feature Engineering
test_desc_list = test["Canonical_Smiles"].apply(smiles_to_descriptors)
test_desc_df = pd.DataFrame(test_desc_list.tolist(), columns=[
    "MolWt", "MolLogP", "HDonors", "HAcceptors", "TPSA", "RotBonds", "Rings"
])

# ✅ 학습 및 검증 데이터 분리
X = desc_df.drop(columns=["Inhibition"])
y = desc_df["Inhibition"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Optuna 튜닝 함수 (verbose 제거된 버전)
def objective(trial):
    param = {
        "n_estimators": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 2.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 2.0),
        "random_state": 42
    }

    model = lgb.LGBMRegressor(**param)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[early_stopping(50)]
    )

    val_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, val_pred)
    rmse = np.sqrt(mse)
    return rmse



# ✅ Optuna 튜닝 실행
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)
print("🎯 Best trial:", study.best_trial.params)

# ✅ 최적 파라미터로 재학습
best_params = study.best_trial.params
best_params["n_estimators"] = 1000
best_params["random_state"] = 42

final_model = lgb.LGBMRegressor(**best_params)
final_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[early_stopping(50), log_evaluation(100)]
)

# ✅ 검증 RMSE 확인
val_preds = final_model.predict(X_val)
mse = mean_squared_error(y_val, val_preds)  # 🔄 squared=False 제거
rmse = np.sqrt(mse)
print(f"✅ 최적 모델 Validation RMSE: {rmse:.4f}")


# ✅ 테스트셋 예측 및 제출 파일 저장
test_preds = final_model.predict(test_desc_df)
submission["Inhibition"] = test_preds
submission.to_csv("submission.csv", index=False)
print("📁 submission.csv 파일 생성 완료!")


✅ 압축 해제 완료: /content/data_1


[I 2025-06-30 08:05:45,375] A new study created in memory with name: no-name-0cd66b26-ba4e-4b94-847d-5db2c37065cd
[I 2025-06-30 08:05:45,436] Trial 0 finished with value: 24.61642401446602 and parameters: {'learning_rate': 0.17097523500133743, 'max_depth': 10, 'num_leaves': 69, 'subsample': 0.6067201047519967, 'colsample_bytree': 0.9570035088497643, 'reg_alpha': 0.5687774616579648, 'reg_lambda': 0.17578438675359442}. Best is trial 0 with value: 24.61642401446602.
[I 2025-06-30 08:05:45,488] Trial 1 finished with value: 24.993819395671952 and parameters: {'learning_rate': 0.11664208600852242, 'max_depth': 8, 'num_leaves': 71, 'subsample': 0.979516774922504, 'colsample_bytree': 0.8033056382322894, 'reg_alpha': 1.1382380776898227, 'reg_lambda': 0.9829972734407848}. Best is trial 0 with value: 24.61642401446602.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000141 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 806
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 7
[LightGBM] [Info] Start training from score 33.391242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[I 2025-06-30 08:05:45,609] Trial 2 finished with value: 25.070508297198593 and parameters: {'learning_rate': 0.02257513863614744, 'max_depth': 9, 'num_leaves': 95, 'subsample': 0.7325130221444422, 'colsample_bytree': 0.7623122270723213, 'reg_alpha': 1.0324991862743336, 'reg_lambda': 1.1107559089812191}. Best is trial 0 with value: 24.61642401446602.
[I 2025-06-30 08:05:45,646] Trial 3 finished with value: 25.060134926751395 and parameters: {'learning_rate': 0.1301231834090336, 'max_depth': 5, 'num_leaves': 82, 'subsample': 0.7426314482425685, 'colsample_bytree': 0.7223512617868167, 'reg_alpha': 0.15694873339508297, 'reg_lambda': 0.051527152332071724}. Best is trial 0 with value: 24.61642401446602.
[I 2025-06-30 08:05:45,689] Trial 4 finished with value: 24.462468303526553 and parameters: {'learning_rate': 0.05171132455952218, 'max_depth': 3, 'num_leaves': 113, 'subsample': 0.6631762708475286, 'colsample_bytree': 0.6198265665016284, 'reg_alpha': 0.013292932062950236, 'reg_lambda': 1.70

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[62]	valid_0's l2: 628.53
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000144 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 806
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 7
[LightGBM] [Info] Start training from score 33.391242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000161 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 806
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 7
[LightGBM] [Info] Start training from score 33.391242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[I 2025-06-30 08:05:47,629] Trial 26 finished with value: 24.908522052346235 and parameters: {'learning_rate': 0.05530842335328117, 'max_depth': 5, 'num_leaves': 55, 'subsample': 0.7161391164417685, 'colsample_bytree': 0.9672514004740279, 'reg_alpha': 1.2379465402384673, 'reg_lambda': 0.7675307908194566}. Best is trial 4 with value: 24.462468303526553.
[I 2025-06-30 08:05:47,708] Trial 27 finished with value: 24.940526476935474 and parameters: {'learning_rate': 0.16111546865645254, 'max_depth': 6, 'num_leaves': 74, 'subsample': 0.6632495258295066, 'colsample_bytree': 0.9088861853526052, 'reg_alpha': 0.9108853238771937, 'reg_lambda': 1.8421852583175642}. Best is trial 4 with value: 24.462468303526553.
[I 2025-06-30 08:05:47,782] Trial 28 finished with value: 24.857993238777546 and parameters: {'learning_rate': 0.1029388308708725, 'max_depth': 4, 'num_leaves': 103, 'subsample': 0.8545118910454033, 'colsample_bytree': 0.6416046597067198, 'reg_alpha': 0.7199529223447159, 'reg_lambda': 1.28

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000149 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 806
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 7
[LightGBM] [Info] Start training from score 33.391242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[I 2025-06-30 08:05:47,879] Trial 29 finished with value: 24.961080054598924 and parameters: {'learning_rate': 0.12282145192183158, 'max_depth': 7, 'num_leaves': 126, 'subsample': 0.9593128570692172, 'colsample_bytree': 0.8030752717907491, 'reg_alpha': 1.1425386170194975, 'reg_lambda': 1.019713017702899}. Best is trial 4 with value: 24.462468303526553.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000155 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 806
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 7
[LightGBM] [Info] Start training from score 33.391242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

In [24]:
from google.colab import files
files.download("submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:

# ✅ 라이브러리 임포트
import zipfile, os, warnings
import pandas as pd
import numpy as np
import optuna
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
warnings.filterwarnings("ignore")

# ✅ 데이터 압축 해제
zip_path = "/content/drive/MyDrive/data.zip"
extract_path = "/content/data_1"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"✅ 압축 해제 완료: {extract_path}")

# ✅ 경로 함수
def get_path(filename):
    return os.path.join(extract_path, filename)

# ✅ 데이터 로딩 및 컬럼 정리
train_df = pd.read_csv(get_path("train.csv"))
test_df = pd.read_csv(get_path("test.csv"))
submission_df = pd.read_csv(get_path("sample_submission.csv"))

train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
submission_df.columns = submission_df.columns.str.strip()

# ✅ Morgan Fingerprint 추출 함수
def get_morgan_fingerprint(smiles_string, n_bits=2048, radius=2):
    try:
        mol = Chem.MolFromSmiles(smiles_string)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
            return np.array(fp)
        else:
            return np.zeros(n_bits, dtype=int)
    except:
        return np.zeros(n_bits, dtype=int)

# ✅ Feature Engineering: Fingerprint 생성
print("🔄 Morgan Fingerprint 생성 중...")
train_fps = train_df['Canonical_Smiles'].apply(get_morgan_fingerprint)
test_fps = test_df['Canonical_Smiles'].apply(get_morgan_fingerprint)

fp_columns = [f'FP_{i}' for i in range(2048)]
train_fp_df = pd.DataFrame(np.vstack(train_fps.values), columns=fp_columns)
test_fp_df = pd.DataFrame(np.vstack(test_fps.values), columns=fp_columns)

X = train_fp_df
y = train_df["Inhibition"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Optuna 튜닝 함수
def objective(trial):
    param = {
        "n_estimators": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 2.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 2.0),
        "random_state": 42
    }

    model = lgb.LGBMRegressor(**param)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[early_stopping(50)]
    )

    val_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, val_pred)
    rmse = np.sqrt(mse)
    return rmse

# ✅ Optuna 튜닝 실행
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)
print("🎯 Best trial params:", study.best_trial.params)

# ✅ 최적 파라미터로 최종 모델 학습
best_params = study.best_trial.params
best_params["n_estimators"] = 1000
best_params["random_state"] = 42

final_model = lgb.LGBMRegressor(**best_params)
final_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[early_stopping(50), log_evaluation(100)]
)

# ✅ 검증 RMSE 확인
val_preds = final_model.predict(X_val)
mse = mean_squared_error(y_val, val_preds)
rmse = np.sqrt(mse)
print(f"✅ 최종 모델 Validation RMSE: {rmse:.4f}")

# ✅ 테스트 예측 + 제출 파일 저장
test_preds = final_model.predict(test_fp_df)
submission_df["Inhibition"] = np.clip(test_preds, 0, 100)
submission_df.to_csv("submission_morgan_optuna.csv", index=False)
print("📁 submission_morgan_optuna.csv 파일 생성 완료!")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007224 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1208
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 604
[LightGBM] [Info] Start training from score 33.391242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

[I 2025-06-30 08:09:52,195] Trial 27 finished with value: 24.777083248023946 and parameters: {'learning_rate': 0.15959626225083576, 'max_depth': 8, 'num_leaves': 104, 'subsample': 0.6662499248793339, 'colsample_bytree': 0.9160162051557573, 'reg_alpha': 0.8133581425780139, 'reg_lambda': 1.077407220547896}. Best is trial 3 with value: 24.399226434215638.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012153 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1208
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 604
[LightGBM] [Info] Start training from score 33.391242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

[I 2025-06-30 08:09:52,539] Trial 28 finished with value: 24.846094919362532 and parameters: {'learning_rate': 0.10303134356164664, 'max_depth': 10, 'num_leaves': 74, 'subsample': 0.7555341614504386, 'colsample_bytree': 0.6324328645844374, 'reg_alpha': 1.8049080880422768, 'reg_lambda': 1.7404476346216762}. Best is trial 3 with value: 24.399226434215638.
[I 2025-06-30 08:09:52,777] Trial 29 finished with value: 24.592036257363027 and parameters: {'learning_rate': 0.1918854100868682, 'max_depth': 5, 'num_leaves': 122, 'subsample': 0.7191197571093882, 'colsample_bytree': 0.7247683945251474, 'reg_alpha': 0.41865600564863403, 'reg_lambda': 1.2615661853276963}. Best is trial 3 with value: 24.399226434215638.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007287 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1208
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 604
[LightGBM] [Info] Start training from score 33.391242
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

In [26]:
from google.colab import files
files.download("submission_morgan_optuna.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>